# **Comparison of SOC Estimation Techniques for Lithium-Ion Batteries**
## A sandbox aimed at gaining hands-on experience in assessing different methodologies and drawbacks.

In [1]:
# %load_ext autoreload
# %autoreload 2

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

import plotly.graph_objects as go
from plotly.subplots import make_subplots

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

<h3> Simulate a single phase </h3>

In [2]:
from utils.notebook_utils import create_search_widget, create_dynamic_line_plots, create_single_fig_line_plot
from utils.data_generator import PyBAMMDataset

In [3]:
#====================================
PyBAMM_DS = PyBAMMDataset(
    capacity_Ah = 5, # Ah,
    initial_soc_percent = 10,
    cc_demand_A = -2,
    evaluation_time_step = 10, # s
    num_points = 1000 # datapoints
)
#====================================

sim = PyBAMM_DS.generate_sim_single_phase()

<h4> Options to choose from for visualization of variables from sim </h4>

<h5> Parameter settings that can be changed vefore solving a sim </h5>

In [4]:
create_search_widget(sim.parameter_values)

Text(value='', description='Search:', layout=Layout(width='300px'), placeholder='Type to search...')

Output()

<h5> Variables that can be called from sim solution to visualize </h5>

In [5]:
create_search_widget(sim.model.variable_names())

Text(value='', description='Search:', layout=Layout(width='300px'), placeholder='Type to search...')

Output()

<h4> Load data into variables for visualization + visualize </h4>

In [6]:
entries_of_interest  = [
    "Time [s]",
    "Terminal voltage [V]",
    "Current [A]",
    "Cell temperature [K]",
    "Negative electrode stoichiometry",
    "Battery open-circuit voltage [V]"
]

tup = PyBAMM_DS.extract_entries_from_sim(entries_of_interest)

In [7]:
create_dynamic_line_plots(tup[1:], tup[0])

## SOC Estimation Methodology Comparison
<h4> Coulomb Counting </h4>

In [8]:
from models.soc_estimation import StateEstimator
SOCEstimator = StateEstimator(PyBAMM_DS)

OCV-SOC mapping located/retrieved from 'c:\Users\Javaid Baksh\Documents\Projects\battery-state-estimation\datasets\ocv_soc_mapping_pybamm_5Ah.pkl'


In [9]:
soc_by_coulcnt = SOCEstimator.coulomb_counting()
create_single_fig_line_plot([soc_by_coulcnt], PyBAMM_DS.data.time, "SOC Estimation")

<h4> Linear KF </h4>

Prediction Steps
1. State Prediction Time Update (x_pred)
- This is to determine an expectation of the next SOC that we expect to achieve from a previous SOC state after a current is applied across a certain timestep (dt).
- In our case, we essentially coulomb count.

2. Error Covariance Time Update (P_pred)
- This quantifies the uncertainty in the predicted state.
- It propagates previous error covariance thourhg state trnasiiotn mode (A) and accounts for process noise covariance (Q)
- This in turn reflexcts uncertainty of the model itself.
        
3. Predict System Output (y_pred)
- We determine predicted measurement based on predicted state.
- In our case we use a predicted SOC to determine a predicted OCV.
        
Correction Steps

4. Estimator Gain Matrix (K)
- We want to determine how much weight to give new measurement vs predicted.
- We use predicted erro covariance and measurement noise covariance (R).
- Higher gain = More trust in measurement relative to prediction.
        
5. State Update with Measurement
-  We can now update state estimate by incorporating our actual measurement z, in our case is "actual" OCV.
- (z - y_pred) is innovation or measurement residual.
- Kalman gain scales residual to adjust predicted SOC.

6. Error Variance Measurement Update
- Update error covariance amtrix to reflext both new measurement + associated uncertainty.

In [10]:
entries_of_interest  = [
    "Battery open-circuit voltage [V]"
]
tup = PyBAMM_DS.extract_entries_from_sim(entries_of_interest)

PyBAMM_DS.update_sim_vector(
    ocv = tup[0]
)

In [11]:
soc_by_KF = SOCEstimator.linKF_OCVSOC(
    P_initial=np.array([[0]]), 
    Q=np.array([[1e-3]]), 
    R=np.array([[1e-3]])
)
# print(soc_by_KF)
create_single_fig_line_plot([soc_by_KF, soc_by_coulcnt], PyBAMM_DS.data.time, "SOC Estimation")

In [12]:
others_to_plot = []
for k,v in soc_by_KF["other"].items():
    try:
        v = [arr.item() for arr in v]
    except: pass
    others_to_plot.append({"label":k, "data":v})
create_single_fig_line_plot(others_to_plot[1:], others_to_plot[0], "SOC Estimation")
